
## Sitio Web de Referencia
[Bolsa de Trabajo de Apple México](https://jobs.apple.com/en-us/search?location=mexico-MEXC)

## Tareas a Realizar

- **Scraping de Páginas Principales**: Obtener una lista de las categorías de trabajo disponibles en la bolsa de trabajo de Apple México. Extraer el enlace y el nombre de cada categoría. 

- **Scraping de Ofertas de Trabajo:** Seleccionar una categoría de trabajo específica (por ejemplo, "Ingeniería"). Extraer información de al menos 5 ofertas de trabajo, incluyendo título, ubicación, fecha de publicación y descripción

- **Manejo de Paginación:** Si la bolsa de trabajo tiene múltiples páginas, desarrollar un script que maneje la paginación y extraiga información de todas las páginas.

- **Datos Adicionales:** Para cada oferta de trabajo, intentar extraer información adicional como requisitos y responsabilidades (si está disponible). 

- **Procesamiento y almacenamiento de Datos:** Procesar la información según los parametros necesarios para la plataforma (ETL). Almacenar la información extraída en un formato estructurado como lo es un archivo .json.

In [1]:
## Importación de librerias
import json
import pandas as pd
import numpy as np
from bs4  import BeautifulSoup
import requests

## Punto - 1 Scraping de Páginas Principales

In [40]:
# funcion para scraping de paginas principales, por categorias de trabajo
#recibe la url a scrapear (website) y devuelve las categorias de trabajo con nombre y link  correspondiente.
def categorias_trabajo(website):
    ## creo una lista vacia para guardar categorias
    categories = []
    
    # realizo el request a la pagina web de trabajo y obtengo los links que contienen las categorias
    response = requests.get(website)
    
    # verifico si  la peticion fue exitosa
    if response.status_code  == 200:
    # parseo el html de la pagina con BeautifulSoup
        soup = BeautifulSoup(response.content,"html.parser")
    
        #busco todos los elementos que contengan categorias de trabajo
        category_elements = soup.find_all("a", class_= "table--advanced-search__title")
        #itero sobre los elementos para extraer nombre y enlace de categoria
        for category in  category_elements:
            category_name = category.text.strip()   #elimino espacios en blanco alrededor del texto
            category_link = "https://jobs.apple.com" + category["href"]
            # agrego cada categoria a mi lista de categorias
            categories.append({"name":category_name , "link":category_link})
    
    return categories

In [41]:
#url del sitio de apple filtrado por pais MEX
website = "https://jobs.apple.com/en-us/search?location=mexico-MEXC"
#llamado a la funcion
categories= categorias_trabajo(website)
for category in categories:
    print(f"Nombre: {category['name']}, Enlace: {category['link']}")

Nombre: MX-Operations Expert, Enlace: https://jobs.apple.com/en-us/details/114438285/mx-operations-expert?team=APPST
Nombre: MX-Genius, Enlace: https://jobs.apple.com/en-us/details/114438291/mx-genius?team=APPST
Nombre: MX-Business Pro, Enlace: https://jobs.apple.com/en-us/details/200125422/mx-business-pro?team=APPST
Nombre: MX-Technical Specialist, Enlace: https://jobs.apple.com/en-us/details/114438287/mx-technical-specialist?team=APPST
Nombre: MX-Creative, Enlace: https://jobs.apple.com/en-us/details/114438295/mx-creative?team=APPST
Nombre: MX-Business Expert, Enlace: https://jobs.apple.com/en-us/details/114438292/mx-business-expert?team=APPST
Nombre: MX-Expert, Enlace: https://jobs.apple.com/en-us/details/114438290/mx-expert?team=APPST
Nombre: MX - Specialist: Full-Time, Part-Time, and Part-Time Temporary, Enlace: https://jobs.apple.com/en-us/details/114438297/mx-specialist-full-time-part-time-and-part-time-temporary?team=APPST
Nombre: Channel Account Executive - SMB and Carriers, E

In [42]:
# guardar las categorías en un archivo JSON
with open('Archivos json/categories.json', 'w') as json_file:
    json.dump(categories, json_file, indent=4)

print("Categorías guardadas en categories.json")

Categorías guardadas en categories.json


## Punto - 2 Scraping de Ofertas de Trabajo

In [72]:
# funcion para scraping de ofertas de trabajo, por categorias de trabajo específica
#recibe la url a scrapear (website) y devuelve según la categoria de trabajo, Título, Ubicación, Fecha de publicación y descripción del empleo.
def ofertas_trabajo(website_categoria):
    ## creo una lista vacia para guardar categorias
    job_offers = []
    
    # realizo el request a la pagina web de trabajo y obtengo los links que contienen las categorias
    response = requests.get(website_categoria)
    
    # verifico si  la peticion fue exitosa
    if response.status_code  == 200:
    # parseo el html de la pagina con BeautifulSoup
        soup = BeautifulSoup(response.content,"html.parser")
    
        #busco todos los elementos que contengan categorias de trabajo
        offer_elements = soup.find_all("a", class_= "table--advanced-search__title")
        
        #itero sobre los elementos para extraer nombre y enlace de categoria
        # limito a solo 5 ofertas por ahora (utilizo filtro en página por categoria)
        for offer in  offer_elements[:5]:
            offer_title = offer.text.strip()   #elimino espacios en blanco alrededor del texto
            offer_link = "https://jobs.apple.com" + offer["href"]

        #buscar la ubicación y fecha de publicación dentro de las etiquetas span
            offer_location_elem = offer.find_next_sibling("span", class_="table--advanced-search__role")
            offer_date_elem = offer_location_elem.find_next_sibling("span",class_="table--advanced-search__date") if offer_location_elem else None
            
        #verifico si se encontro ubicación y fecha para luego acceder al link 
            offer_location = offer_location_elem.text.strip() if offer_location_elem else "No disponible"
            offer_date = offer_date_elem.text.strip() if offer_date_elem else "No disponible"
            
        # ingresar en el link extraido para obtener la descripcion de la oferta
        #se realiza nuevamente un request en el nuevo link
            offer_response= requests.get(offer_link)
            if offer_response.status_code ==  200 :
                offer_soup = BeautifulSoup(offer_response.content, "html.parser")
                offer_description_elem =  offer_soup.find("div", id ="jd-description",class_="jd__row--main jd__summary--main")
                offer_description = offer_description_elem.text.strip() if offer_description_elem else"No disponible"

            # agrego cada oferta de trabajo a mi lista de ofertas
            job_offers.append({"title": offer_title, "location": offer_location, "date": offer_date, "description": offer_description})
    return job_offers

In [73]:
website_categoria= "https://jobs.apple.com/en-us/search?location=mexico-MEXC&team=sales-APPST-ARSS+support-APPST-ARSCS"

#llamo a la función 
job_offers= ofertas_trabajo(website_categoria)
#imprimo por separado los datos de la oferta para ordenar la salida
for offer in job_offers:
    print(f"Título: {offer['title']}")
    print(f"Ubicación: {offer['location']}")
    print(f"Fecha de publicación: {offer['date']}")
    print(f"Descripción: {offer['description']}")
    print()

Título: MX-Operations Expert
Ubicación: Apple Retail
Fecha de publicación: Feb 16, 2024
Descripción: As an Operations Expert, you and your team have the incredible responsibility of ensuring products take the final step in the supply chain: getting into customers' hands. You're in charge of the store's entire inventory - products, parts, tools, supplies, and everything else. You make sure your team has the support, knowledge, and resources required to maintain product availability, complete inventory tasks, and keep the stockroom organized as new products arrive. You're in constant contact with the management and leadership teams, sharing data about the status of products and parts. And when exciting new products arrive, you're the first to open them up and present them to the entire store team. Apple makes the products, but you make it happen by being ready to place our products in customers' hands.

Título: MX-Business Pro
Ubicación: Apple Retail
Fecha de publicación: Feb 16, 2024
De

In [74]:
# guardar las categorías en un archivo JSON
with open('Archivos json/job_offers.json', 'w') as json_file:
    json.dump(job_offers, json_file, indent=4)

print("Ofertas de Trabajo guardadas en job_offers.json")

Ofertas de Trabajo guardadas en job_offers.json


## Punto - 3 Manejo de paginación

In [35]:

def paginacion(web):
    job_offers = []

    page = 1
    while True:
        # concateno el número de página a la url
        url = f"{web}&page={page}"

        #realizo la peticion
        response = requests.get(url)

        # corroboro si la solicitud fue exitosa 
        if response.status_code == 200:
            # creo un objeto BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Busco a través de las etiquetas, las ofertas de trabajo
            offer_elements = soup.find_all('a', class_='table--advanced-search__title')

            # Si no hay ofertas de trabajo en la página, salir del bucle
            if not offer_elements:
                break

            # itero sobre los elementos encontrados y extraer información de las ofertas de trabajo
            for offer in offer_elements:
                offer_title = offer.text.strip()
                offer_link = "https://jobs.apple.com" + offer['href']
                job_offers.append({'title': offer_title, 'link': offer_link})

            # Incrementar el número de página para la siguiente solicitud
            page += 1
        else:
            # Si la solicitud no fue exitosa, salir del bucle
            break

    return job_offers

#  ejecuto la función con la URL correspondiente a Apple
web = "https://jobs.apple.com/en-us/search?sort=relevance&homeOffice=true"

# llamo a la función 
job_offers = paginacion(web)

# imprimo las ofertas de trabajo obtenidas
for offer in job_offers:
    print(f"Título: {offer['title']}, Enlace: {offer['link']}")


Título: SAP Business Governance - Security Liaison, Enlace: https://jobs.apple.com/en-us/details/200535238/sap-business-governance-security-liaison?team=OPMFG
Título: Apple Support College Program At Home Advisor - University of North Carolina at Wilmington, Enlace: https://jobs.apple.com/en-us/details/200493551/apple-support-college-program-at-home-advisor-university-of-north-carolina-at-wilmington?team=STDNT
Título: Apple Support College Program At Home Advisor - Appalachian State University, Enlace: https://jobs.apple.com/en-us/details/200493539/apple-support-college-program-at-home-advisor-appalachian-state-university?team=STDNT
Título: ER Business Partner (Retail), Enlace: https://jobs.apple.com/en-us/details/200537633/er-business-partner-retail?team=CORSV
Título: Apple Support College Program At Home Advisor - California State University, Fresno, Enlace: https://jobs.apple.com/en-us/details/200493591/apple-support-college-program-at-home-advisor-california-state-university-fresno

In [38]:
# guardar las categorías en un archivo JSON
with open('Archivos json/pages.json', 'w') as json_file:
    json.dump(job_offers, json_file, indent=4)

print("Ofertas de Trabajo guardadas en pages.json")

Ofertas de Trabajo guardadas en pages.json


## Punto - 4 Datos Adicionales 

In [114]:
# funcion para scraping de ofertas de trabajo, por categorias de trabajo específica
#recibe la url a scrapear (website) y devuelve según la categoria de trabajo, Título, Ubicación, Fecha de publicación , descripción del empleo y datos adicionales .
def adicionales(web):
    ## creo una lista vacia para guardar categorias
    jobs = []
    
    # realizo el request a la pagina web de trabajo y obtengo los links que contienen las categorias
    response = requests.get(website_categoria)
    
    # verifico si  la peticion fue exitosa
    if response.status_code  == 200:
    # parseo el html de la pagina con BeautifulSoup
        soup = BeautifulSoup(response.content,"html.parser")
    
        #busco todos los elementos que contengan categorias de trabajo
        offer_elements = soup.find_all("a", class_= "table--advanced-search__title")
        
        #itero sobre los elementos para extraer nombre y enlace de categoria
        # limito a solo 5 ofertas por ahora (utilizo filtro en página por categoria)
        for offer in  offer_elements[:5]:
            offer_title = offer.text.strip()   #elimino espacios en blanco alrededor del texto
            offer_link = "https://jobs.apple.com" + offer["href"]

        #buscar la ubicación y fecha de publicación dentro de las etiquetas span
            offer_location_elem = offer.find_next_sibling("span", class_="table--advanced-search__role")
            offer_date_elem = offer_location_elem.find_next_sibling("span",class_="table--advanced-search__date") if offer_location_elem else None
            
        #verifico si se encontro ubicación y fecha para luego acceder al link 
            offer_location = offer_location_elem.text.strip() if offer_location_elem else "No disponible"
            offer_date = offer_date_elem.text.strip() if offer_date_elem else "No disponible"
            
        # ingresar en el link extraido para obtener la descripcion de la oferta
        #se realiza nuevamente un request en el nuevo link
            offer_response= requests.get(offer_link)
            if offer_response.status_code ==  200 :
                offer_soup = BeautifulSoup(offer_response.content, "html.parser")
                offer_description_elem =  offer_soup.find("div", id ="jd-description",class_="jd__row--main jd__summary--main")
                offer_description = offer_description_elem.text.strip() if offer_description_elem else"No disponible"

        # busco los elementos que contienen los requisitos adicionales y habilidades
                requirements_elem = offer_soup.find("div", id ="jd-key-qualifications", class_="jd__row--main jd__summary")
                skills_elem = offer_soup.find("div", id ="jd-additional-requirements",class_="jd__row--main jd__summary")

        # extraigo el texto de los elementos encontrados 
                key_qualifications = requirements_elem.text.strip() if requirements_elem else "No disponible"
                additional= skills_elem.text.strip() if skills_elem else "No disponible"
            else:
                key_qualifications = "No disponible"
                additional = "No disponible"


        # agrego cada oferta de trabajo a mi lista de ofertas
            jobs.append({
                "title": offer_title, 
                "location": offer_location, 
                "date": offer_date, 
                "description": offer_description,
                "key_qualifications": key_qualifications,
                "additionals": additional
                })
    return jobs

In [115]:
web= "https://jobs.apple.com/en-us/search?location=mexico-MEXC"

#llamo a la función 
jobs = adicionales(web)
#imprimo por separado los datos de la oferta para ordenar la salida
for offer in jobs:
    print(f"Título: {offer['title']}")
    print(f"Ubicación: {offer['location']}")
    print(f"Fecha de publicación: {offer['date']}")
    print(f"Descripción: {offer['description']}")
    print(f"Aspectos Clave: {offer['key_qualifications']}")
    print(f"Adicionales: {offer['additionals']}")
    print()

Título: MX-Business Pro
Ubicación: Apple Retail
Fecha de publicación: Feb 16, 2024
Descripción: You lead customer engagement, deepen relationships, drive sales and awareness of Apple’s value proposition for business and build loyalty with a focus on high potential business customers. You plan and forecast business performance through account management, pipeline building, and opportunity management, utilizing CRM and other tools. You are passionate and confident and engage business customers by showcasing our technology and helping them discover how Apple and third-party solutions and services can transform the way they work. You develop effective account plans to start and expand adoption of Apple solutions across our ecosystem platform and deeper in the customer’s organization.  You lead and support briefings and leverage workshops to support customer engagement. You effectively use CRMs to build and maintain accurate customer account information, manage relationships, opportunities,

In [110]:
# guardar las datos adicionales en un archivo JSON
with open('Archivos json/datos_adicionales.json', 'w') as json_file:
    json.dump(jobs, json_file, indent=4)

print("Ofertas de Trabajo con datos adicionales guardadas en datos_adicionales.json")

Ofertas de Trabajo con datos adicionales guardadas en datos_adicionales.json


## Punto - 5 Crear un modelo de almacenamiento.

In [140]:
import requests
from bs4 import BeautifulSoup
from typing import List, Optional
from pydantic import BaseModel, Extra

class Skill(BaseModel):
    name: str
    level: str
    experience: int

class Aptitude(BaseModel):
    name: str

class Tool(BaseModel):
    name: str
    level: str
    experience: int

class Language(BaseModel):
    name: str
    level: str

class Benefit(BaseModel):
    name: str

class JobPost(BaseModel):
    name: Optional[str] = None
    location: Optional[str]= None
    date: Optional[str] = None
    description: Optional[str] = None
    key_qualifications: Optional[str]=None
    additionals: Optional[str]= None


def adicionales(website_categoria: str) -> List[JobPost]:
    job_posts = []
    response = requests.get(website_categoria)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        offer_elements = soup.find_all("a", class_= "table--advanced-search__title")

        for offer in offer_elements[:5]:
            # Aquí iría la lógica para extraer los datos de la oferta de trabajo
            # y crear instancias de JobPost, Skill, Aptitude, Tool, Language, Benefit
            # y agregarlos a las listas correspondientes en el JobPost.
            # Una vez que hayas creado las instancias, agregas el JobPost a la lista job_posts.
            offer_title = offer.text.strip()   #elimino espacios en blanco alrededor del texto
            offer_link = "https://jobs.apple.com" + offer["href"]

        #buscar la ubicación y fecha de publicación dentro de las etiquetas span
            offer_location_elem = offer.find_next_sibling("span", class_="table--advanced-search__role")
            offer_date_elem = offer_location_elem.find_next_sibling("span",class_="table--advanced-search__date") if offer_location_elem else None
            
        #verifico si se encontro ubicación y fecha para luego acceder al link 
            offer_location = offer_location_elem.text.strip() if offer_location_elem else "No disponible"
            offer_date = offer_date_elem.text.strip() if offer_date_elem else "No disponible"
            
        # ingresar en el link extraido para obtener la descripcion de la oferta
        #se realiza nuevamente un request en el nuevo link
            offer_response= requests.get(offer_link)
            if offer_response.status_code ==  200 :
                offer_soup = BeautifulSoup(offer_response.content, "html.parser")
                offer_description_elem =  offer_soup.find("div", id ="jd-description",class_="jd__row--main jd__summary--main")
                offer_description = offer_description_elem.text.strip() if offer_description_elem else"No disponible"

        # busco los elementos que contienen los requisitos adicionales y habilidades
                requirements_elem = offer_soup.find("div", id ="jd-key-qualifications", class_="jd__row--main jd__summary")
                skills_elem = offer_soup.find("div", id ="jd-additional-requirements",class_="jd__row--main jd__summary")

        # extraigo el texto de los elementos encontrados 
                key_qualifications = requirements_elem.text.strip() if requirements_elem else "No disponible"
                additional= skills_elem.text.strip() if skills_elem else "No disponible"
            else:
                key_qualifications = "No disponible"
                additional = "No disponible"

            # agrego cada oferta de trabajo a mi lista de ofertas
            job_posts.append(JobPost(
                name= offer_title, 
                location= offer_location, 
                date= offer_date, 
                description= offer_description,
                key_qualifications= key_qualifications,
                additionals= additional
            ))
    return job_posts




In [141]:
website_categoria = "https://jobs.apple.com/en-us/search?location=mexico-MEXC"
job_offers = adicionales(website_categoria)

# Serializar la lista de instancias de JobPost a JSON
json_output = [job.model_dump_json() for job in job_offers]
print(json_output)

['{"name":"MX-Business Pro","location":"Apple Retail","date":"Feb 16, 2024","description":"You lead customer engagement, deepen relationships, drive sales and awareness of Apple’s value proposition for business and build loyalty with a focus on high potential business customers. You plan and forecast business performance through account management, pipeline building, and opportunity management, utilizing CRM and other tools. You are passionate and confident and engage business customers by showcasing our technology and helping them discover how Apple and third-party solutions and services can transform the way they work. You develop effective account plans to start and expand adoption of Apple solutions across our ecosystem platform and deeper in the customer’s organization.  You lead and support briefings and leverage workshops to support customer engagement. You effectively use CRMs to build and maintain accurate customer account information, manage relationships, opportunities, task

In [142]:
json_output

['{"name":"MX-Business Pro","location":"Apple Retail","date":"Feb 16, 2024","description":"You lead customer engagement, deepen relationships, drive sales and awareness of Apple’s value proposition for business and build loyalty with a focus on high potential business customers. You plan and forecast business performance through account management, pipeline building, and opportunity management, utilizing CRM and other tools. You are passionate and confident and engage business customers by showcasing our technology and helping them discover how Apple and third-party solutions and services can transform the way they work. You develop effective account plans to start and expand adoption of Apple solutions across our ecosystem platform and deeper in the customer’s organization.  You lead and support briefings and leverage workshops to support customer engagement. You effectively use CRMs to build and maintain accurate customer account information, manage relationships, opportunities, task

In [144]:
def guardar_en_modelo(job_offers: List[JobPost], filename: str):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("from pydantic import BaseModel, Extra\n\n")

        f.write("class JobPost(BaseModel):\n")
        f.write("    name: str\n")
        f.write("    description: str\n")
        f.write("    key_qualifications: str\n")
        f.write("    additionals: str\n")
        f.write("    location: str\n")
        f.write("    date: str\n")
        # Agregar los otros campos definidos en el modelo JobPost

        for job in job_offers:
            f.write(f"\n\n{job.name.replace(' ', '')} = JobPost(\n")
            f.write(f"    name='{job.name}',\n")
            f.write("    description='" + job.description.replace("'", r"\\'").replace('\n', '\\n') + "',\n")
            f.write("    key_qualifications='" + job.key_qualifications.replace("'", r"\\'").replace('\n', '\\n') + "',\n")
            f.write("    additionals='" + job.additionals.replace("'", r"\\'").replace('\n', '\\n') + "',\n")
            f.write(f"    location='{job.location}',\n")
            f.write(f"    date='{job.date}'\n")
            # Agregar los otros campos definidos en el modelo JobPost
            f.write(")\n")

website_categoria = "https://jobs.apple.com/en-us/search?location=mexico-MEXC"
job_offers = adicionales(website_categoria)
guardar_en_modelo(job_offers, "modelo.py")
